## 1. Setup

In [1]:
import sys
import os
import h5py
import gc
from glob import glob
from pathlib import Path
from functools import lru_cache
from collections import defaultdict
from fastprogress import progress_bar
from copy import deepcopy
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image, ExifTags
import cv2
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
import torch
import torch.nn.functional as F
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import kornia
import pycolmap

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
competition_dataset = Path('/kaggle/input/image-matching-challenge-2023')
external_dataset = Path('/kaggle/input/image-matching-challenge-2023-dataset')

In [3]:
sys.path.append(str(external_dataset / 'packages' / 'SuperGluePretrainedNetwork'))
from models.matching import Matching

In [4]:
df = pd.read_csv(competition_dataset / 'sample_submission.csv')

if df.shape[0] != 3:
    # Enable submission mode and disable verbose
    submission = True
    verbose = False
    train_or_test_directory = competition_dataset / 'test'
else:
    # Disable submission mode and enable verbose
    submission = False
    verbose = True
    train_or_test_directory = competition_dataset / 'train'

if submission is False:
    df = pd.read_csv(competition_dataset / 'train' / 'train_labels.csv')
    # Use only bike scene from the haiper dataset if it's not submission mode
    df = df.loc[df['scene'] == 'bike', :]
    
print(f'Dataset Shape: {df.shape}')

Dataset Shape: (15, 5)


In [5]:
def load_angle_detection_model():
    
    """
    Load angle detection model and processor
    
    Returns
    -------
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    """
    
    vit = TFViTModel.from_pretrained(external_dataset / 'models' / 'vit-base-patch16-224')
    
    input_layer = Input(shape=(3, 224, 224))
    x = vit(input_layer)
    y = Dense(1, activation='linear')(x[-1])
    
    model = Model(input_layer, y)
    model.load_weights(external_dataset / 'models' / 'deep-image-orientation-angle-detection' / 'model-vit-ang-loss.h5')
    
    processor = ViTFeatureExtractor.from_pretrained(external_dataset / 'models' / 'vit-base-patch16-224')
    
    return model, processor


def detect_angle(image, model, processor):
    
    """
    Detect angle of a given image
    
    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, channel)
        Image array
        
    model: keras.engine.functional.Functional
        Angle detection model
        
    processor: transformers.ViTFeatureExtractor
        Image processor
    
    Returns
    -------
    outputs: float
    """
    
    inputs = processor(images=[image], return_tensors='np')['pixel_values']
    outputs = model.predict(inputs, verbose=None)[0][0]
        
    return outputs


In [6]:
#angle_detection_model, angle_detection_processor = load_angle_detection_model()

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

    image_path = train_or_test_directory / row['image_path']
    image = np.array(Image.open(image_path))
    
    # Extract image dimensions and memory usage
    df.loc[idx, 'image_height'] = image.shape[0]
    df.loc[idx, 'image_width'] = image.shape[1]
    df.loc[idx, 'memory_usage'] = image.nbytes
    
    # Extract image orientation
    '''
    try:
        with tf.device('/cpu:0'):
            angle = detect_angle(
                image=image,
                model=angle_detection_model,
                processor=angle_detection_processor
            )
            df.loc[idx, 'angle'] = angle
    except:
        df.loc[idx, 'angle'] = 0
    ''' 


df['image_id'] = df['image_path'].apply(lambda x: str(x).split('/')[-1])
df['memory_usage'] /= (1024 ** 2)

df['image_height'] = df['image_height'].astype(np.uint16)
df['image_width'] = df['image_width'].astype(np.uint16)
df['memory_usage'] = df['memory_usage'].astype(np.float32)
'''
df['angle'] = df['angle'].astype(np.float32)
angles = df[['image_path', 'angle']].set_index('image_path').to_dict()['angle']
del angle_detection_model, angle_detection_processor
gc.collect()
torch.cuda.empty_cache()
K.clear_session()
'''

100%|██████████| 15/15 [00:00<00:00, 27.38it/s]


"\ndf['angle'] = df['angle'].astype(np.float32)\nangles = df[['image_path', 'angle']].set_index('image_path').to_dict()['angle']\ndel angle_detection_model, angle_detection_processor\ngc.collect()\ntorch.cuda.empty_cache()\nK.clear_session()\n"

In [7]:
df

,dataset,scene,image_path,rotation_matrix,translation_vector,image_height,image_width,memory_usage,image_id
230,haiper,bike,haiper/bike/images/image_004.jpeg,-0.0444086367008516;-0.9943622104926982;0.0962...,-0.1960672197513899;-1.7201514631645476;2.9038...,1920,1440,7.910156,image_004.jpeg
231,haiper,bike,haiper/bike/images/image_029.jpeg,-0.30379429789458934;-0.780428110344117;0.5464...,-0.7743795049636117;-2.4063622468195978;4.2039...,1920,1440,7.910156,image_029.jpeg
232,haiper,bike,haiper/bike/images/image_038.jpeg,-0.44075875074949056;-0.6141344795467536;0.654...,-0.8061619977689489;-1.3614914099692277;2.5759...,1920,1440,7.910156,image_038.jpeg
233,haiper,bike,haiper/bike/images/image_049.jpeg,-0.4615070783507189;-0.037215312681071866;0.88...,-0.05029237007998646;-2.0953351075544067;3.128...,1920,1440,7.910156,image_049.jpeg
234,haiper,bike,haiper/bike/images/image_062.jpeg,0.06454466684801141;0.9943063402602297;0.08478...,0.45813698932578173;-1.7352377000311676;3.6787...,1920,1440,7.910156,image_062.jpeg
235,haiper,bike,haiper/bike/images/image_076.jpeg,0.566876909470875;-0.09510152245268588;-0.8182...,1.1496049380742543;-1.4930263438651057;2.57378...,1920,1440,7.910156,image_076.jpeg
236,haiper,bike,haiper/bike/images/image_088.jpeg,0.253507028584536;-0.8760074072730466;-0.41029...,-0.44871563031823725;-1.750899302580256;2.6802...,1920,1440,7.910156,image_088.jpeg
237,haiper,bike,haiper/bike/images/image_094.jpeg,0.15438718127182738;-0.9293944893965761;-0.335...,-0.37595896679394963;-2.714322519243274;4.1322...,1920,1440,7.910156,image_094.jpeg
238,haiper,bike,haiper/bike/images/image_101.jpeg,0.4234658773801474;-0.6722258122106511;-0.6072...,-0.11276896793699917;-0.7117396520115244;1.914...,1920,1440,7.910156,image_101.jpeg
239,haiper,bike,haiper/bike/images/image_115.jpeg,0.3904043657227909;0.16367189264027227;-0.9059...,0.8296975499797659;-2.3480417943432315;3.47250...,1920,1440,7.910156,image_115.jpeg


## 2. Image Utilities

In [8]:
def get_global_desc(image_paths, model, device):
        
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    
    global_descs = []
    for i, img_fname_full in tqdm(enumerate(image_paths),total=len(image_paths)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        img = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                desc = model.forward_features(img.to(device)).mean(dim=(-1, 2))
            desc = desc.view(1, -1).detach().cpu().float()
            desc = F.normalize(desc, dim=1, p=2)
        global_descs.append(desc)
    global_descs = torch.cat(global_descs, dim=0)
    
    return global_descs


def create_image_pairs_exhaustive(image_paths):

    """
    Create all possible image pairs from given list of image paths

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    Returns
    -------
    image_pair_indices: list of shape (n_image_pairs)
        List of tuples of image pair indices
    """

    image_pair_indices = []

    for i in range(len(image_paths)):
        for j in range(i + 1, len(image_paths)):
            image_pair_indices.append((i, j))

    return image_pair_indices


def create_image_pairs(image_paths, sim_th=0.6, min_pairs=20, exhaustive_if_less=20, device=torch.device('cuda')):
    
    n_images = len(image_paths)

    if n_images <= exhaustive_if_less:
        
        image_pair_indices = []
        # Create all possible image pairs from given list of image paths
        for i in range(len(image_paths)):
            for j in range(i + 1, len(image_paths)):
                image_pair_indices.append((i, j))

        return image_pair_indices
    
    else:

        model = timm.create_model(
            'tf_efficientnet_b7',
            checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b7/1/tf_efficientnet_b7_ra-6c08e654.pth'
        )
        model = model.eval().to(image_matching_device)
        descs = get_global_desc(image_paths, model, device=device)
        dm = torch.cdist(descs, descs, p=2).numpy()
        mask = dm <= sim_th
        total = 0
        image_pair_indices = []
        ar = np.arange(n_images)
        already_there_set = []
        for st_idx in range(n_images - 1):
            mask_idx = mask[st_idx]
            to_match = ar[mask_idx]
            if len(to_match) < min_pairs:
                to_match = np.argsort(dm[st_idx])[:min_pairs]  
            for idx in to_match:
                if st_idx == idx:
                    continue
                if dm[st_idx, idx] < 1000:
                    image_pair_indices.append(tuple(sorted((st_idx, idx.item()))))
                    total += 1
                    
        image_pair_indices = sorted(list(set(image_pair_indices)))
        
        return image_pair_indices


def resize_with_aspect_ratio(image, longest_edge):

    """
    Resize image while preserving its aspect ratio

    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, 3)
        Image array

    longest_edge: int
        Desired number of pixels on the longest edge

    Returns
    -------
    image: numpy.ndarray of shape (resized_height, resized_width, 3)
        Resized image array
    """

    height, width = image.shape[:2]
    scale = longest_edge / max(height, width)
    image = cv2.resize(image, dsize=(int(width * scale), int(height * scale)), interpolation=cv2.INTER_LANCZOS4)

    return image


def get_image_tensor(image_path_or_array, resize, resize_shape, resize_longest_edge, scale, grayscale):

    """
    Load image and preprocess it

    Parameters
    ----------
    image_path_or_array: str or numpy.ndarray of shape (height, width, 3)
        Image path or image array

    resize: bool
        Whether to resize the image or not

    resize_shape: tuple or int
        Tuple of image height and width or number of pixels for both height and width

    resize_longest_edge: bool
        Whether to resize the longest edge or not

    scale: bool
        Whether to scale image pixel values by max 8-bit pixel value or not

    grayscale: bool
        Whether to convert RGB image to grayscale or not

    Returns
    -------
    image: torch.Tensor of shape (1, 1 or 3, height, width)
        Image tensor
    """

    if isinstance(image_path_or_array, Path) or isinstance(image_path_or_array, str):
        # Read image from the given path if image_path_or_array is a path-like string
        image = cv2.imread(str(image_path_or_array))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path_or_array

    if resize:
        if resize_longest_edge:
            image = resize_with_aspect_ratio(image=image, longest_edge=resize_shape)
        else:
            resize_shape = (resize_shape, resize_shape) if isinstance(resize_shape, int) else resize_shape
            image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LANCZOS4)

    if scale:
        image = image / 255.

    image = kornia.image_to_tensor(image, False).float()
    if grayscale:
        image = kornia.color.rgb_to_grayscale(image)

    return image


def crop(image, keypoints, perc_points=0.85, pad=5):

    norm_keypoints = MinMaxScaler().fit_transform(keypoints)
    total = len(keypoints)
    best_dist = 1
    best_clusters = None
    best_asm = None
    for eps in [0.01, 0.025, 0.05, 0.1, 0.2]:
        clusters = DBSCAN(eps=eps).fit_predict(norm_keypoints)
        counts = pd.Series(clusters).value_counts().sort_values(ascending=False)
        counts = counts[counts.index > -1]
        if len(counts) == 0:
            continue

        cumsums = np.cumsum(counts.values) / total
        dists = np.abs(cumsums - perc_points)
        best_ix = np.argmin(dists)

        if dists[best_ix] < best_dist:
            best_dist = dists[best_ix]
            best_clusters = list(counts.head(best_ix + 1).index)
            best_asm = clusters

    mask = np.isin(best_asm, best_clusters)

    miny = int(np.min(keypoints[mask][:, 1]))
    miny = max(miny - pad, 0)

    maxy = int(np.max(keypoints[mask][:, 1]))
    maxy = min(maxy + pad, image.shape[0])

    minx = int(np.min(keypoints[mask][:, 0]))
    minx = max(minx - pad, 0)

    maxx = int(np.max(keypoints[mask][:, 0]))
    maxx = min(maxx + pad, image.shape[1])

    image_cropped = image[miny:maxy + 1, minx:maxx + 1, :]
    keypoints_cropped = np.copy(keypoints)
    keypoints_cropped[:, 0] -= minx
    keypoints_cropped[:, 1] -= miny

    keypoints_cropped = keypoints_cropped[(keypoints_cropped[:, 0] > 0) & (keypoints_cropped[:, 1] > 0) & (keypoints_cropped[:, 0] < image_cropped.shape[1]) & (keypoints_cropped[:, 1] < image_cropped.shape[0])]

    return image_cropped, keypoints_cropped, minx, miny


def rotate_image(img, angle):
    height, width = img.shape[:2]
    matrix = cv2.getRotationMatrix2D((width / 2 - 0.5, height / 2 - 0.5), angle, 1.0)
    matrix = np.vstack((matrix, [[0, 0, 1]]))
    return cv2.warpPerspective(img, matrix, (width, height))


def rotate_coordinates(img, angle, keypoints):
    height, width = img.shape[:2]
    inv_matrix = cv2.getRotationMatrix2D((width / 2 - 0.5, height / 2 - 0.5), angle, 1.0)
    inv_matrix = np.vstack((inv_matrix, [[0, 0, 1]]))
    return cv2.perspectiveTransform(keypoints[None, :, :], inv_matrix)[0]


## 3. Camera Utilities

In [9]:
def get_focal_length(image_path):

    """
    Get focal length from EXIF or calculate it using prior

    Parameters
    ----------
    image_path: str
        Image path

    Returns
    -------
    focal_length: float
        Focal length extracted from EXIF or calculated using prior
    """

    image = Image.open(image_path)
    image_longest_edge = max(image.size)

    focal_length = None
    exif = image.getexif()
    
    try:
        if image._getexif() is not None:
            exif = {
                ExifTags.TAGS[k]: v
                for k, v in image._getexif().items()
                if k in ExifTags.TAGS
            }
        else:
            exif = None
    except:
        exif = None
       
    if exif is not None:

        focal_length_35mm = None

        for tag, value in exif.items():
            if tag == 'FocalLengthIn35mmFilm':
                focal_length_35mm = float(value)
        
        if focal_length_35mm is not None:
            focal_length = focal_length_35mm / 35. * image_longest_edge

    if focal_length is None:
        prior_focal_length = 1.2
        focal_length = prior_focal_length * image_longest_edge

    return focal_length


## 4. Database Utilities

In [10]:
MAX_IMAGE_ID = 2 ** 31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    return array.tostring()


def blob_to_array(blob, dtype, shape=(-1,)):
    return np.fromstring(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))


In [11]:
def create_camera(db, image_path, camera_model):
    
    image = Image.open(image_path)
    width, height = image.size

    focal_length = get_focal_length(image_path)

    if camera_model == 'simple-pinhole':
        model = 0
        param_arr = np.array([focal_length, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1
        param_arr = np.array([focal_length, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2
        param_arr = np.array([focal_length, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4
        param_arr = np.array([focal_length, focal_length, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, camera_model, single_camera=True):
    
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')
    camera_id = None
    fname_to_id = {}
    
    for filename in tqdm(list(keypoint_f.keys())):
        
        keypoints = keypoint_f[filename][()]

        path = os.path.join(image_path, filename)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id


def add_matches(db, h5_path, fname_to_id):
    
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)
                added.add(pair_id)
                pbar.update(1)
                
                
def get_unique_idxs(A, dim=0):
    
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    
    return first_indices


## 5. Evaluation

In [12]:
def array_to_string(a):

    """
    Flatten given array and convert it to a string with semicolon delimiters

    Parameters
    ----------
    a: np.ndarray
        N-dimensional array

    Returns
    -------
    s: string
        String form of the given array
    """

    s = ';'.join([str(x) for x in a.reshape(-1)])

    return s


def string_to_array(s):

    """
    Convert semicolon delimited string to an array

    Parameters
    ----------
    s: string
        String form of the array

    Returns
    -------
    a: np.ndarray
        N-dimensional array
    """

    a = np.array(s.split(';')).astype(np.float64)

    return a


def rotation_matrix_to_quaternion(rotation_matrix):

    """
    Convert rotation matrix to quaternion

    Parameters
    ----------
    rotation_matrix: numpy.ndarray of shape (3, 3)
        Array of directions of the world-axes in camera coordinates

    Returns
    -------
    quaternion: numpy.ndarray of shape (4)
        Array of quaternion
    """

    r00 = rotation_matrix[0, 0]
    r01 = rotation_matrix[0, 1]
    r02 = rotation_matrix[0, 2]
    r10 = rotation_matrix[1, 0]
    r11 = rotation_matrix[1, 1]
    r12 = rotation_matrix[1, 2]
    r20 = rotation_matrix[2, 0]
    r21 = rotation_matrix[2, 1]
    r22 = rotation_matrix[2, 2]

    k = np.array([
        [r00 - r11 - r22, 0.0, 0.0, 0.0],
        [r01 + r10, r11 - r00 - r22, 0.0, 0.0],
        [r02 + r20, r12 + r21, r22 - r00 - r11, 0.0],
        [r21 - r12, r02 - r20, r10 - r01, r00 + r11 + r22]
    ])
    k /= 3.0

    # Quaternion is the eigenvector of k that corresponds to the largest eigenvalue
    w, v = np.linalg.eigh(k)
    quaternion = v[[3, 0, 1, 2], np.argmax(w)]

    if quaternion[0] < 0:
        np.negative(quaternion, quaternion)

    return quaternion


def pose_difference(r1, t1, r2, t2):

    """
    Calculate relative pose difference from given rotation matrices and translation vectors

    Parameters
    ----------
    r1: numpy.ndarray of shape (3, 3)
        First rotation matrix

    t1: numpy.ndarray of shape (3)
        First translation vector

    r2: numpy.ndarray of shape (3, 3)
        Second rotation matrix

    t2: numpy.ndarray of shape (3)
        Second translation vector

    Returns
    -------
    rotation_difference: float
        Rotation difference in terms of degrees from the first image

    translation_difference: float
        Translation difference in terms of meters from the first image
    """

    rotation_difference = np.dot(r2, r1.T)
    translation_difference = t2 - np.dot(rotation_difference, t1)

    return rotation_difference, translation_difference


def rotation_and_translation_error(q_ground_truth, t_ground_truth, q_prediction, t_prediction, epsilon=1e-15):

    """
    Calculate rotation and translation error

    Parameters
    ----------
    q_ground_truth: numpy.ndarray of shape (4)
        Array of quaternion derived from ground truth rotation matrix

    t_ground_truth: numpy.ndarray of shape (3)
        Array of ground truth translation vector

    q_prediction: numpy.ndarray of shape (4)
        Array of quaternion derived from estimated rotation matrix

    t_prediction: numpy.ndarray of shape (3)
        Array of estimated translation vector

    epsilon: float
        A small number for preventing zero division

    Returns
    -------
    rotation_error: float
        Rotation error in terms of degrees

    translation_error: float
        Translation error in terms of meters
    """

    q_ground_truth_norm = q_ground_truth / (np.linalg.norm(q_ground_truth) + epsilon)
    q_prediction_norm = q_prediction / (np.linalg.norm(q_prediction) + epsilon)
    loss_q = np.maximum(epsilon, (1.0 - np.sum(q_prediction_norm * q_ground_truth_norm) ** 2))

    rotation_error = np.degrees(np.arccos(1 - (2 * loss_q)))

    scaling_factor = np.linalg.norm(t_ground_truth)
    t_prediction = scaling_factor * (t_prediction / (np.linalg.norm(t_prediction) + epsilon))
    translation_error = min(
        np.linalg.norm(t_ground_truth - t_prediction),
        np.linalg.norm(t_ground_truth + t_prediction)
    )

    return rotation_error, translation_error


def mean_average_accuracy(rotation_errors, translation_errors, rotation_error_thresholds, translation_error_thresholds):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    rotation_errors: list of shape (n_pairs)
        List of rotation errors

    translation_errors: list of shape (n_pairs)
        List of translation errors

    rotation_error_thresholds: numpy.ndarray of shape (10)
        Array of rotation error thresholds

    translation_error_thresholds: numpy.ndarray of shape (10)
        Array of translation error thresholds

    Returns
    -------
    maa: float
        Mean average accuracy calculated on both rotation and translation errors

    rotation_maa: float
        Mean average accuracy calculated on rotation errors

    translation_maa: float
        Mean average accuracy calculated on translation errors
    """

    accuracies, rotation_accuracies, translation_accuracies = [], [], []

    for rotation_error_threshold, translation_error_threshold in zip(rotation_error_thresholds, translation_error_thresholds):

        # Calculate whether the errors are less than specified thresholds or not
        rotation_accuracy = (rotation_errors <= rotation_error_threshold)
        translation_accuracy = (translation_errors <= translation_error_threshold)
        accuracy = rotation_accuracy & translation_accuracy

        accuracies.append(accuracy.astype(np.float32).mean())
        rotation_accuracies.append(rotation_accuracy.astype(np.float32).mean())
        translation_accuracies.append(translation_accuracy.astype(np.float32).mean())

    maa = np.array(accuracies).mean()
    rotation_maa = np.array(rotation_accuracies).mean()
    translation_maa = np.array(translation_accuracies).mean()

    return maa, rotation_maa, translation_maa


def evaluate(df, verbose=False):

    """
    Calculate mean average accuracies over a set of thresholds for rotation and translation

    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe with dataset, scene, rotation_matrix, translation_vector, rotation_matrix_prediction and translation_vector_prediction columns

    verbose: bool
        Whether to print scores or not

    Returns
    -------
    df_scores: pandas.DataFrame
        Dataframe of scores
    """

    rotation_error_thresholds = {
        **{('haiper', scene): np.linspace(1, 10, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.linspace(1, 10, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.linspace(0.2, 10, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.linspace(1, 10, 10)},
    }
    translation_error_thresholds = {
        **{('haiper', scene): np.geomspace(0.05, 0.5, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.geomspace(0.1, 2, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.geomspace(0.05, 1, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.geomspace(0.5, 5, 10)},
    }
    df_scores = pd.DataFrame(columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa'])

    for (dataset, scene), df_scene in tqdm(df.groupby(['dataset', 'scene'])):

        scene_rotation_errors = []
        scene_translation_errors = []

        for i in range(df_scene.shape[0]):
            for j in range(i + 1, df_scene.shape[0]):

                rotation_matrix_difference_ground_truth, translation_vector_difference_ground_truth = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector'])),
                )
                quaternion_ground_truth = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_ground_truth)

                rotation_matrix_difference_prediction, translation_vector_difference_prediction = pose_difference(
                    r1=string_to_array((df_scene.iloc[i]['rotation_matrix_prediction'])).reshape(3, 3),
                    t1=string_to_array((df_scene.iloc[i]['translation_vector_prediction'])),
                    r2=string_to_array((df_scene.iloc[j]['rotation_matrix_prediction'])).reshape(3, 3),
                    t2=string_to_array((df_scene.iloc[j]['translation_vector_prediction'])),
                )
                quaternion_prediction = rotation_matrix_to_quaternion(rotation_matrix=rotation_matrix_difference_prediction)

                rotation_error, translation_error = rotation_and_translation_error(
                    q_ground_truth=quaternion_ground_truth,
                    t_ground_truth=translation_vector_difference_ground_truth,
                    q_prediction=quaternion_prediction,
                    t_prediction=translation_vector_difference_prediction,
                    epsilon=1e-15
                )
                scene_rotation_errors.append(rotation_error)
                scene_translation_errors.append(translation_error)

        scene_maa, scene_rotation_maa, scene_translation_maa = mean_average_accuracy(
            rotation_errors=scene_rotation_errors,
            translation_errors=scene_translation_errors,
            rotation_error_thresholds=rotation_error_thresholds[(dataset, scene)],
            translation_error_thresholds=translation_error_thresholds[(dataset, scene)]
        )

        if verbose:
            settings.logger.info(
                f'''
                Dataset: {dataset} - Scene: {scene}
                Number of image pairs: {len(scene_rotation_errors)}
                mAA: {scene_maa:.6f} - rotation mAA: {scene_rotation_maa:.6f} - translation mAA: {scene_translation_maa:.6f}
                '''
            )

        df_scores = pd.concat((
            df_scores,
            pd.DataFrame(
                data=[[dataset, scene, len(scene_rotation_errors), scene_maa, scene_rotation_maa, scene_translation_maa]],
                columns=['dataset', 'scene', 'image_pairs', 'maa', 'rotation_maa', 'translation_maa']
            )
        ), axis=0)

    return df_scores


## 6. Models

In [13]:
def loftr_match_images(image1, image2, model, device, amp, transforms, confidence_threshold, top_k):

    """
    Match given two images with each other using LoFTR model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        LoFTR Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    confidence_threshold: float or int
        Confidence threshold to filter out low confidence matches

    top_k: int
        Number of matches to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        outputs[k] = outputs[k].detach().cpu().numpy()

    if confidence_threshold is not None:
        if isinstance(confidence_threshold, float):
            # Select matched keypoints with above given confidence threshold
            confidence_mask = outputs['confidence'] >= confidence_threshold
        elif isinstance(confidence_threshold, int):
            # Select keypoints dynamically based on confidence distribution
            confidence_mean, confidence_std = outputs['confidence'].mean(), outputs['confidence'].std()
            confidence_mask = outputs['confidence'] >= (confidence_mean + (confidence_std * confidence_threshold))
        else:
            raise ValueError(f'Invalid confidence_threshold {confidence_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][confidence_mask]

    if top_k is not None:
        # Select top-k keypoints based on their confidences
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


In [14]:
def superglue_match_images(image1, image2, model, device, amp, transforms, score_threshold, top_k):

    """
    Match given two images with each other using SuperGlue model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Array of first image

    image2: numpy.ndarray of shape (3, height, width)
        Array of second image

    model: torch.nn.Module
        SuperGlue Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    score_threshold: float, int or None
        Confidence threshold

    top_k: int or None
        Number of keypoints to take

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms[1]['resize'],
        resize_shape=transforms[1]['resize_shape'],
        resize_longest_edge=transforms[1]['resize_longest_edge'],
        scale=transforms[1]['scale'],
        grayscale=transforms[1]['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms[2]['resize'],
        resize_shape=transforms[2]['resize_shape'],
        resize_longest_edge=transforms[2]['resize_longest_edge'],
        scale=transforms[2]['scale'],
        grayscale=transforms[2]['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model({'image0': image1, 'image1': image2})
        else:
            outputs = model({'image0': image1, 'image1': image2})

    for k in outputs.keys():
        if k == 'descriptors0' or k == 'descriptors1':
            outputs[k] = outputs[k][0].detach().cpu().numpy().T
        else:
            outputs[k] = outputs[k][0].detach().cpu().numpy()

    matches_mask = outputs['matches0'] > -1

    for k in ['keypoints1', 'scores1', 'descriptors1', 'matches1', 'matching_scores1']:
        outputs[k] = outputs[k][outputs['matches0'][matches_mask]]

    for k in ['keypoints0', 'scores0', 'descriptors0', 'matches0', 'matching_scores0']:
        outputs[k] = outputs[k][matches_mask]

    if score_threshold is not None:
        if isinstance(score_threshold, float):
            # Select matched keypoints with above given score threshold
            score_mask = outputs['matching_scores0'] >= score_threshold
        elif isinstance(score_threshold, int):
            # Select keypoints dynamically based on score distribution
            score_mean, score_std = outputs['matching_scores0'].mean(), outputs['matching_scores0'].std()
            score_mask = outputs['matching_scores0'] >= (score_mean + (score_std * score_threshold))
        else:
            raise ValueError(f'Invalid score_threshold {score_threshold}')

        for k in outputs.keys():
            outputs[k] = outputs[k][score_mask]

    if top_k is not None:
        # Select top-k keypoints based on their scores
        sorting_idx = outputs['matching_scores0'].argsort()[-top_k:]
        for k in outputs.keys():
            outputs[k] = outputs[k][sorting_idx]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


## 7. Inference

In [15]:
image_matching_device = torch.device('cuda')
'''
# Load LoFTR model with specified configurations
loftr_model = LoFTR(
    pretrained=None,
    config={
        'backbone_type': 'ResNetFPN',
        'resolution': (8, 2),
        'fine_window_size': 5,
        'fine_concat_coarse_feat': True,
        'resnetfpn': {
            'initial_dim': 128,
            'block_dims': [128, 196, 256]
        },
        'coarse': {
            'd_model': 256,
            'd_ffn': 256,
            'nhead': 8,
            'layer_names': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'],
            'attention': 'linear',
            'temp_bug_fix': False,
        },
        'match_coarse': {
            'thr': 0.2,
            'border_rm': 2,
            'match_type': 'dual_softmax',
            'dsmax_temperature': 0.1,
            'skh_iters': 3,
            'skh_init_bin_score': 1.0,
            'skh_prefilter': True,
            'train_coarse_percent': 0.4,
            'train_pad_num_gt_min': 200,
        },
        'fine': {
            'd_model': 128,
            'd_ffn': 128,
            'nhead': 8,
            'layer_names': ['self', 'cross'],
            'attention': 'linear'
        }
    }
)
loftr_model.load_state_dict(torch.load(external_dataset / 'models' / 'loftr' / 'loftr_outdoor.ckpt')['state_dict'], strict=False)
loftr_model = loftr_model.eval().to(image_matching_device)
'''
# Load SuperPoint and SuperGlue model with specified configurations
superglue_model = Matching(config={
    'superpoint': {
        'descriptor_dim': 256,
        'nms_radius': 4,
        'keypoint_threshold': 0.01,
        'max_keypoints': -1,
        'remove_borders': 4
    },
    'superglue': {
        'descriptor_dim': 256,
        'weights': 'outdoor',
        'keypoint_encoder': [32, 64, 128, 256],
        'sinkhorn_iterations': 100,
        'match_threshold': 0.2
    }
})
superglue_model = superglue_model.eval().to(image_matching_device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [16]:
def match(read_image_function, image_paths, image_pair_indices, feature_dir, superglue_model, stage2=False):
    
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='w') as f_match:
        for pair_idx in progress_bar(image_pair_indices):
            
            idx1, idx2 = pair_idx
            fname1, fname2 = image_paths[idx1], image_paths[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            
            image1 = read_image_function(fname1)
            image2 = read_image_function(fname2)
            
            keypoints1 = []
            keypoints2 = []
            
            # Largest SuperGlue size that can be used on Kaggle GPU is 2560
            superglue_longest_edge_limit = 2560
            if (np.max(image1.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2.shape[:2]) > superglue_longest_edge_limit):
                # Both of the images have longest edges greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    }
                }
            elif (np.max(image1.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2.shape[:2]) <= superglue_longest_edge_limit):
                # First image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    }
                }
            elif (np.max(image1.shape[:2]) <= superglue_longest_edge_limit) and (np.max(image2.shape[:2]) > superglue_longest_edge_limit):
                # Second image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': True,
                        'resize_shape': superglue_longest_edge_limit,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True
                    }
                }
            else:
                # Neither of the image's longest edge is greater than 2560
                first_stage_superglue_transforms = {
                    1: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    },
                    2: {
                        'resize': False,
                        'resize_shape': None,
                        'resize_longest_edge': None,
                        'scale': True,
                        'grayscale': True
                    }
                }
                
            first_stage_superglue_outputs = superglue_match_images(
                image1=image1,
                image2=image2,
                model=superglue_model,
                device=image_matching_device,
                amp=True,
                transforms=first_stage_superglue_transforms,
                score_threshold=None,
                top_k=None
            )
            
            keypoints1.append(first_stage_superglue_outputs['keypoints0'])
            keypoints2.append(first_stage_superglue_outputs['keypoints1'])
            del first_stage_superglue_outputs
            
            keypoints1 = np.concatenate(keypoints1, axis=0)
            keypoints2 = np.concatenate(keypoints2, axis=0)
            
            if stage2:
                if (keypoints1.shape[0] > 10) & (keypoints1.shape[0] < 200):
                    
                    image1_cropped, keypoints1_cropped, x_offset1, y_offset1 = crop(image1, keypoints1)
                    image2_cropped, keypoints2_cropped, x_offset2, y_offset2 = crop(image2, keypoints2)
                    
                    # Largest SuperGlue size that can be used on Kaggle GPU is 2560
                    superglue_longest_edge_limit = 2560
                    if (np.max(image1_cropped.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2_cropped.shape[:2]) > superglue_longest_edge_limit):
                        # Both of the images have longest edges greater than 2560
                        second_stage_superglue_transforms = {
                            1: {
                                'resize': True,
                                'resize_shape': superglue_longest_edge_limit,
                                'resize_longest_edge': True,
                                'scale': True,
                                'grayscale': True
                            },
                            2: {
                                'resize': True,
                                'resize_shape': superglue_longest_edge_limit,
                                'resize_longest_edge': True,
                                'scale': True,
                                'grayscale': True
                            }
                        }
                    elif (np.max(image1_cropped.shape[:2]) > superglue_longest_edge_limit) and (np.max(image2_cropped.shape[:2]) <= superglue_longest_edge_limit):
                        # First image's longest edge is greater than 2560
                        second_stage_superglue_transforms = {
                            1: {
                                'resize': True,
                                'resize_shape': superglue_longest_edge_limit,
                                'resize_longest_edge': True,
                                'scale': True,
                                'grayscale': True
                            },
                            2: {
                                'resize': False,
                                'resize_shape': None,
                                'resize_longest_edge': None,
                                'scale': True,
                                'grayscale': True
                            }
                        }
                    elif (np.max(image1_cropped.shape[:2]) <= superglue_longest_edge_limit) and (np.max(image2_cropped.shape[:2]) > superglue_longest_edge_limit):
                        # Second image's longest edge is greater than 2560
                        second_stage_superglue_transforms = {
                            1: {
                                'resize': False,
                                'resize_shape': None,
                                'resize_longest_edge': None,
                                'scale': True,
                                'grayscale': True
                            },
                            2: {
                                'resize': True,
                                'resize_shape': superglue_longest_edge_limit,
                                'resize_longest_edge': True,
                                'scale': True,
                                'grayscale': True
                            }
                        }
                    else:
                        # Neither of the image's longest edge is greater than 2560
                        second_stage_superglue_transforms = {
                            1: {
                                'resize': False,
                                'resize_shape': None,
                                'resize_longest_edge': None,
                                'scale': True,
                                'grayscale': True
                            },
                            2: {
                                'resize': False,
                                'resize_shape': None,
                                'resize_longest_edge': None,
                                'scale': True,
                                'grayscale': True
                            }
                        }

                    second_stage_superglue_outputs = superglue_match_images(
                        image1=image1_cropped,
                        image2=image2_cropped,
                        model=superglue_model,
                        device=image_matching_device,
                        amp=True,
                        transforms=second_stage_superglue_transforms,
                        score_threshold=None,
                        top_k=200
                    )

                    second_stage_superglue_outputs['keypoints0'][:, 0] += x_offset1
                    second_stage_superglue_outputs['keypoints0'][:, 1] += y_offset1
                    second_stage_superglue_outputs['keypoints1'][:, 0] += x_offset2
                    second_stage_superglue_outputs['keypoints1'][:, 1] += y_offset2

                    keypoints1 = np.concatenate([keypoints1, second_stage_superglue_outputs['keypoints0']], axis=0)
                    keypoints2 = np.concatenate([keypoints2, second_stage_superglue_outputs['keypoints1']], axis=0)
            
            n_matches = len(keypoints1)
            group  = f_match.require_group(key1)
            if n_matches >= 15:
                group.create_dataset(key2, data=np.concatenate([keypoints1, keypoints2], axis=1))

    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0] += total_kpts[k1]
                current_match[:, 1] += total_kpts[k2]
                total_kpts[k1] += len(matches)
                total_kpts[k2] += len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
        
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
        
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([
                unique_kpts[k1][m2[:,0]],
                unique_kpts[k2][m2[:,1]]
            ], axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
            
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
                
    return


def import_into_colmap(img_dir, feature_dir='.featureout', database_path='colmap.db'):
    
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, 'simple-radial', single_camera)
    add_matches(db, feature_dir, fname_to_id,)
    db.commit()
    
    return


In [17]:
reconstruction_root_directory = Path('./inference')
reconstruction_root_directory.mkdir(parents=True, exist_ok=True)

datasets = [
    directory for directory in os.listdir(train_or_test_directory)
    if (train_or_test_directory / directory).is_dir() and (directory in df['dataset'].unique())
]

for dataset in datasets:
        
    dataset_directory = train_or_test_directory / dataset
    scenes = [
        directory for directory in os.listdir(dataset_directory)
        if (dataset_directory / directory).is_dir() and (directory in df['scene'].unique())
    ]
    
    for scene in scenes:
        
        df_scene = df.loc[df['scene'] == scene, :]
                
        scene_directory = dataset_directory / scene
        scene_image_directory = scene_directory / 'images'
        image_paths = sorted(glob(str(scene_image_directory / '*')))
        scene_image_count = len(image_paths)
        
        scene_reconstruction_directory = reconstruction_root_directory / dataset / scene
        scene_reconstruction_directory.mkdir(parents=True, exist_ok=True)                    
        database_path = scene_reconstruction_directory / 'colmap.db'
        if os.path.isfile(database_path):
            os.remove(database_path)
        
        scene_mean_memory_usage = df_scene['memory_usage'].mean()
        scene_mean_memory_usage_limit = 16
        if scene_mean_memory_usage < scene_mean_memory_usage_limit:
            
            # Create image pair indices from given image paths
            scene_image_pair_indices = create_image_pairs(
                image_paths,
                sim_th=0.85,
                min_pairs=120,
                exhaustive_if_less=200,
                device=image_matching_device
            )
            
            @lru_cache(maxsize=2)
            def read_image(image_path):
                return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
            
            match(
                read_image,
                image_paths,
                scene_image_pair_indices,
                feature_dir=scene_reconstruction_directory,
                superglue_model=superglue_model,
                stage2=False
            )
            import_into_colmap(scene_image_directory, feature_dir=scene_reconstruction_directory, database_path=database_path)
        else:
            sift_extraction_options = pycolmap.SiftExtractionOptions()
            sift_extraction_options.num_threads = -1
            sift_extraction_options.max_image_size = 1400
            sift_extraction_options.max_num_features = 8192
            sift_extraction_options.estimate_affine_shape = False
            sift_extraction_options.upright = False
            sift_extraction_options.normalization = 'L2'
            pycolmap.extract_features(
                database_path=database_path,
                image_path=scene_image_directory,
                image_list=[image_path.split('/')[-1] for image_path in image_paths],
                sift_options=sift_extraction_options,
                device=pycolmap.Device('cpu'),
                verbose=verbose
            )
    
        pycolmap.match_exhaustive(database_path)
        
        output_path = scene_reconstruction_directory / 'colmap_rec'
        output_path.mkdir(parents=True, exist_ok=True)                    

        incremental_mapper_options = pycolmap.IncrementalMapperOptions()
        incremental_mapper_options.min_model_size = 3
        incremental_mapper_options.min_num_matches = 5
        reconstructions = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=scene_image_directory,
            output_path=output_path,
            options=incremental_mapper_options
        )
        
        if len(reconstructions) > 0:

            best_registered_image_count = 0
            best_reconstruction_idx = None

            for reconstruction_idx in reconstructions.keys():
                if reconstructions[reconstruction_idx].num_reg_images() > best_registered_image_count:
                    best_reconstruction_idx = reconstruction_idx
                    best_registered_image_count = reconstructions[reconstruction_idx].num_reg_images()

            best_reconstruction = reconstructions[best_reconstruction_idx]
        else:
            best_registered_image_count = 0
            best_reconstruction_idx = None
            best_reconstruction = None

        if verbose:
            print(
                f'''
                Dataset: {dataset} - Scene: {scene}
                Reconstruction count: {len(reconstructions)}
                Best reconstruction registered image count: {best_registered_image_count}/{scene_image_count}
                '''
            )

        if best_reconstruction is not None:
            registered_images = {image.name: image for image in best_reconstruction.images.values()}
        else:
            registered_images = {}
            
        for idx, row in df.loc[(df['dataset'] == dataset) & (df['scene'] == scene)].iterrows():
            if row['image_id'] in registered_images:
                rotation_matrix_prediction = registered_images[row['image_id']].rotmat()
                translation_vector_prediction = registered_images[row['image_id']].tvec
                df.loc[idx, 'rotation_matrix'] = ';'.join([str(x) for x in rotation_matrix_prediction.reshape(-1)])
                df.loc[idx, 'translation_vector'] = ';'.join([str(x) for x in translation_vector_prediction.reshape(-1)])
            else:
                df.loc[idx, 'rotation_matrix'] = np.nan
                df.loc[idx, 'translation_vector'] = np.nan

        # Fill unregistered images rotation matrices with the prediction mean or zeros
        scene_rotation_matrix_predictions = df.loc[df['scene'] == scene, 'rotation_matrix'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3, 3)).values
        if scene_rotation_matrix_predictions.shape[0] == 0:
            rotation_matrix_fill_value = np.zeros((3, 3))
        else:
            rotation_matrix_fill_value = np.mean(np.concatenate(scene_rotation_matrix_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['rotation_matrix'].isnull()), 'rotation_matrix'] = ';'.join([str(x) for x in rotation_matrix_fill_value.reshape(-1)])

        # Fill unregistered images translation vectors with the prediction mean or zeros
        scene_translation_vector_predictions = df.loc[df['scene'] == scene, 'translation_vector'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3)).values
        if scene_translation_vector_predictions.shape[0] == 0:
            translation_vector_fill_value = np.zeros((3, 1))
        else:
            translation_vector_fill_value = np.mean(np.concatenate(scene_translation_vector_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['translation_vector'].isnull()), 'translation_vector'] = ';'.join([str(x) for x in translation_vector_fill_value.reshape(-1)])


  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_23/2598211829.py:86: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return array.tostring()
 88%|████████▊ | 80/91 [00:00<00:00, 4012.63it/s]



Exhaustive feature matching

Matching block [1/1, 1/1] in 9.628s
Elapsed time: 0.161 [minutes]

Loading database

Loading cameras... 15 in 0.000s
Loading matches... 56 in 0.001s
Loading images... 15 in 0.001s (connected 15)
Building correspondence graph... in 0.004s (ignored 0)

Elapsed time: 0.000 [minutes]


Finding good initial image pair


Initializing with image pair #7 and #1


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.115678e+04    0.00e+00    1.11e+06   0.00e+00   0.00e+00  1.00e+04        0    4.08e-03    1.75e-02
   1  6.897078e+03    4.26e+03    1.07e+06   1.78e+02   4.72e-01  1.00e+04        1    1.05e-02    2.80e-02
   2  1.959483e+03    4.94e+03    7.30e+04   3.96e+01   9.90e-01  3.00e+04        1    6.49e-03    3.46e-02
   3  1.890919e+03    6.86e+01    3.65e+04   1.97e+01   8.61e-01  4.80e+04        1    6.35e-03    4.09e-02
   4  1.876036e+03    1.49e+01    1.62e+04   1

## 8. Submission

In [18]:
df_submission = df.loc[:, ['image_path', 'dataset', 'scene', 'rotation_matrix', 'translation_vector']]
df_submission.to_csv('submission.csv', index=False)